# llcuda v2.2.0 - Kaggle 2× T4 Build Notebook

## Architecture: Split-GPU Workload

```
┌───────────────────────────┬───────────────────────────────┐
│         GPU 0             │            GPU 1              │
│  llama-server (GGUF)      │  RAPIDS + Graphistry          │
│  LLM Inference            │  Graph Visualization (cuGraph)│
│  15GB VRAM                │  15GB VRAM                    │
└───────────────────────────┴───────────────────────────────┘
```

This notebook builds llcuda binaries for **split-GPU** operation:
- **GPU 0**: llama-server with GGUF model (LLM inference)
- **GPU 1**: RAPIDS/Graphistry with cuDF/cuGraph (graph simulation)

## Step 1: Verify Kaggle GPU Environment

In [ ]:
# Verify we have 2× T4 GPUs
import subprocess
import os

print("="*70)
print("KAGGLE GPU ENVIRONMENT CHECK")
print("="*70)

# Check nvidia-smi
result = subprocess.run(["nvidia-smi", "-L"], capture_output=True, text=True)
gpu_lines = [l for l in result.stdout.strip().split("\n") if l.startswith("GPU")]
print(f"\n📊 Detected GPUs: {len(gpu_lines)}")
for line in gpu_lines:
    print(f"   {line}")

# Check CUDA version
print("\n📊 CUDA Version:")
!nvcc --version | grep release

# Check total VRAM
print("\n📊 VRAM Summary:")
!nvidia-smi --query-gpu=index,name,memory.total --format=csv

# Verify we have 2 GPUs
if len(gpu_lines) >= 2:
    print("\n✅ Multi-GPU environment confirmed! Ready for dual-T4 build.")
else:
    print("\n⚠️ WARNING: Less than 2 GPUs detected!")
    print("   Enable 'GPU T4 x2' in Kaggle notebook settings.")

## Step 2: Install Build Dependencies

In [ ]:
%%time
# Install CMake, Ninja, and other build tools
print("Installing build dependencies...")
!apt-get update -qq
!apt-get install -y -qq cmake ninja-build ccache build-essential
!pip install -q huggingface_hub tqdm requests sseclient-py

print("\n✅ Build dependencies installed")
!cmake --version | head -1
!ninja --version

## Step 2b: Install RAPIDS + Graphistry (GPU 1 Workload)

In [ ]:
%%time
# Install RAPIDS (cuDF, cuGraph, cuML) for GPU 1 graph workload
# This runs alongside llama-server on GPU 0

print("="*70)
print("INSTALLING RAPIDS + GRAPHISTRY FOR GPU 1")
print("="*70)

# Install RAPIDS via pip (CUDA 12)
!pip install -q \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12 cuml-cu12 cugraph-cu12 pylibraft-cu12 raft-dask-cu12

# Install Graphistry with GPU support
!pip install -q graphistry[ai]

# Verify installation
print("\n📦 Installed packages:")
import cudf
import cugraph
print(f"   cuDF version: {cudf.__version__}")
print(f"   cuGraph version: {cugraph.__version__}")

try:
    import graphistry
    print(f"   Graphistry version: {graphistry.__version__}")
except:
    print("   Graphistry: available (no __version__)")

print("\n✅ RAPIDS + Graphistry installed for GPU 1")

## Step 3: Clone llama.cpp (Latest Stable)

In [ ]:
%%time
import os

# Set working directory
WORK_DIR = "/kaggle/working"
os.chdir(WORK_DIR)

# Clean any previous build
!rm -rf llama.cpp

# Clone llama.cpp
print("Cloning llama.cpp...")
!git clone --depth 1 https://github.com/ggml-org/llama.cpp.git

os.chdir("llama.cpp")

# Get commit info
print("\n📦 llama.cpp Version:")
!git log -1 --oneline
!git describe --tags --always 2>/dev/null || echo "(no tag)"

## Step 4: Configure CMake for Dual T4 (SM 7.5)

In [ ]:
%%time
import os
os.chdir("/kaggle/working/llama.cpp")

# Clean previous build
!rm -rf build

print("Configuring CMake for Kaggle 2× Tesla T4...")
print("")

# CMake configuration for dual T4
# Key flags:
# - GGML_CUDA=ON: Enable CUDA backend
# - CMAKE_CUDA_ARCHITECTURES=75: Tesla T4 (Turing, SM 7.5)
# - GGML_CUDA_FA_ALL_QUANTS=ON: FlashAttention for ALL quantization types
# - BUILD_SHARED_LIBS=OFF: Static linking for portability
# - GGML_NATIVE=OFF: Don't optimize for build machine (for portability)

cmake_cmd = """
cmake -B build -G Ninja \
    -DGGML_CUDA=ON \
    -DCMAKE_CUDA_ARCHITECTURES="75" \
    -DGGML_CUDA_FA_ALL_QUANTS=ON \
    -DGGML_NATIVE=OFF \
    -DBUILD_SHARED_LIBS=OFF \
    -DLLAMA_BUILD_EXAMPLES=ON \
    -DLLAMA_BUILD_TESTS=OFF \
    -DLLAMA_BUILD_SERVER=ON \
    -DCMAKE_BUILD_TYPE=Release \
    -DCMAKE_C_COMPILER=gcc \
    -DCMAKE_CXX_COMPILER=g++
"""

!{cmake_cmd}

print("\n✅ CMake configuration complete!")
print("   Target: SM 7.5 (Tesla T4)")
print("   FlashAttention: All quantization types")
print("   Static linking: Enabled")

## Step 5: Build llama.cpp (This takes ~8-12 minutes)

In [ ]:
%%time
import os
import multiprocessing
import sys

os.chdir("/kaggle/working/llama.cpp")

# Get CPU count for parallel build
cpu_count = multiprocessing.cpu_count()
print(f"Building with {cpu_count} parallel jobs...")
print("This will take approximately 8-12 minutes.\n")

# Build
build_result = os.system(f"cmake --build build --config Release -j{cpu_count}")

print("\n" + "="*60)

# Verify build succeeded
if build_result == 0 and os.path.exists("build/bin/llama-server"):
    print("✅ BUILD COMPLETE!")
    print("="*60)
    !ls -lh build/bin/llama-server
else:
    print("❌ BUILD FAILED!")
    print("="*60)
    print("Check the build output above for errors.")
    sys.exit(1)

## Step 6: Verify Built Binaries

In [ ]:
import os
os.chdir("/kaggle/working/llama.cpp/build/bin")

print("Built binaries:")
print("="*60)
!ls -lh llama-* 2>/dev/null | head -20

print("\nKey binary sizes:")
!du -h llama-server llama-cli llama-quantize 2>/dev/null

print("\nChecking CUDA support in llama-server:")
!./llama-server --help 2>&1 | grep -i "cuda\|gpu\|ngl" | head -5

## Step 7: Test Multi-GPU Support

In [ ]:
import os
os.chdir("/kaggle/working/llama.cpp/build/bin")

print("Testing multi-GPU CLI flags:")
print("="*60)

# Check for multi-GPU flags
print("\n📌 --tensor-split (VRAM distribution):")
!./llama-server --help 2>&1 | grep -A2 "tensor-split"

print("\n📌 --split-mode (layer/row splitting):")
!./llama-server --help 2>&1 | grep -A2 "split-mode"

print("\n📌 --main-gpu (primary GPU selection):")
!./llama-server --help 2>&1 | grep -A2 "main-gpu"

print("\n✅ Multi-GPU support confirmed!")

## Step 8: Create llcuda v2.2.0 Package

In [ ]:
import os
import shutil
import json
import subprocess
from datetime import datetime

os.chdir("/kaggle/working")

# Package info
VERSION = "2.2.0"
BUILD_DATE = datetime.now().strftime("%Y%m%d")
PACKAGE_NAME = f"llcuda-v{VERSION}-cuda12-kaggle-t4x2"
PACKAGE_DIR = f"/kaggle/working/{PACKAGE_NAME}"

print(f"Creating package: {PACKAGE_NAME}")
print("="*60)

# Create directory structure
os.makedirs(f"{PACKAGE_DIR}/bin", exist_ok=True)
os.makedirs(f"{PACKAGE_DIR}/lib", exist_ok=True)
os.makedirs(f"{PACKAGE_DIR}/include", exist_ok=True)

# Binaries to include
BUILD_BIN = "/kaggle/working/llama.cpp/build/bin"
binaries = [
    # Core server
    "llama-server",
    "llama-cli",
    # Quantization & conversion
    "llama-quantize",
    "llama-gguf",
    "llama-gguf-hash",
    "llama-gguf-split",
    "llama-imatrix",
    # LoRA & embedding
    "llama-export-lora",
    "llama-embedding",
    # Utilities
    "llama-tokenize",
    "llama-infill",
    "llama-perplexity",
    "llama-bench",
    "llama-cvector-generator",
]

# Copy binaries
copied = []
for binary in binaries:
    src = f"{BUILD_BIN}/{binary}"
    if os.path.exists(src):
        shutil.copy2(src, f"{PACKAGE_DIR}/bin/{binary}")
        os.chmod(f"{PACKAGE_DIR}/bin/{binary}", 0o755)
        copied.append(binary)
        print(f"  ✅ {binary}")
    else:
        print(f"  ⚠️  {binary} (not found)")

print(f"\n📦 Copied {len(copied)}/{len(binaries)} binaries")

## Step 9: Create Package Metadata

In [ ]:
import json
import subprocess
from datetime import datetime

# Get llama.cpp info
os.chdir("/kaggle/working/llama.cpp")
commit_hash = subprocess.getoutput("git rev-parse HEAD")
commit_date = subprocess.getoutput("git log -1 --format=%ci")
commit_msg = subprocess.getoutput("git log -1 --format=%s")

# Get CUDA version
cuda_version = subprocess.getoutput("nvcc --version | grep release | sed 's/.*release //' | cut -d, -f1")

# Create metadata
metadata = {
    "package": "llcuda",
    "version": VERSION,
    "build_date": datetime.now().isoformat(),
    "platform": {
        "name": "kaggle",
        "gpu_count": 2,
        "gpu_model": "Tesla T4",
        "vram_per_gpu_gb": 15,
        "total_vram_gb": 30,
        "compute_capability": "7.5",
        "architecture": "Turing"
    },
    "cuda": {
        "version": cuda_version,
        "architectures": ["sm_75"],
        "flash_attention": True,
        "flash_attention_all_quants": True
    },
    "llama_cpp": {
        "commit": commit_hash,
        "commit_date": commit_date,
        "commit_message": commit_msg,
        "repo": "https://github.com/ggml-org/llama.cpp"
    },
    "multi_gpu": {
        "supported": True,
        "method": "native_cuda",
        "modes": {
            "tensor_split": {
                "description": "Split model across both GPUs for larger models",
                "flags": ["--tensor-split 0.5,0.5", "--split-mode layer"],
                "use_case": "Large GGUF models (>15GB)"
            },
            "split_workload": {
                "description": "Dedicated GPU assignment: GPU 0 for LLM, GPU 1 for graphs",
                "method": "CUDA_VISIBLE_DEVICES environment variable",
                "use_case": "LLM inference + RAPIDS/Graphistry graph simulation"
            }
        },
        "recommended_config": {
            "tensor_split": "0.5,0.5",
            "split_mode": "layer",
            "n_gpu_layers": -1
        }
    },
    "split_workload": {
        "description": "Split-GPU architecture for combined LLM + Graph workloads",
        "gpu_0": "llama-server with GGUF model (LLM inference)",
        "gpu_1": "RAPIDS + Graphistry (cuDF, cuGraph for graph visualization)",
        "rapids_packages": ["cudf-cu12", "cuml-cu12", "cugraph-cu12"],
        "graphistry_packages": ["graphistry[ai]"],
        "usage": {
            "llm_gpu": "CUDA_VISIBLE_DEVICES=0 ./llama-server -m model.gguf -ngl 99",
            "graph_gpu": "import os; os.environ['CUDA_VISIBLE_DEVICES']='1'; import cudf, cugraph"
        }
    },
    "binaries": copied,
    "features": [
        "multi-gpu-tensor-split",
        "split-workload-architecture",
        "flash-attention-all-quants",
        "openai-compatible-api",
        "anthropic-compatible-api",
        "29-quantization-formats",
        "lora-adapters",
        "grammar-constraints",
        "json-schema-output",
        "embeddings-reranking",
        "streaming-sse",
        "kv-cache-slots",
        "speculative-decoding"
    ],
    "unsloth_integration": {
        "description": "CUDA 12 inference backend for Unsloth fine-tuned models",
        "workflow": "Unsloth (training) → GGUF (conversion) → llcuda (inference)",
        "supported_exports": ["f16", "q8_0", "q4_k_m", "q5_k_m", "iq4_xs"]
    }
}

# Write metadata
os.chdir("/kaggle/working")
with open(f"{PACKAGE_DIR}/metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("📋 Package Metadata:")
print(json.dumps(metadata, indent=2))

## Step 10: Create README and Usage Guide

In [ ]:
readme_content = f'''# llcuda v{VERSION} - Kaggle 2× Tesla T4 Build

Pre-built CUDA 12 binaries for **Kaggle dual Tesla T4** multi-GPU inference.

## 🎯 Unsloth Integration

llcuda is the **CUDA 12 inference backend for Unsloth**:

```
┌─────────────┐    ┌─────────────┐    ┌─────────────────┐
│   UNSLOTH   │───▶│   LLCUDA    │───▶│  llama-server   │
│  Training   │    │  GGUF Conv  │    │  Multi-GPU Inf  │
│  Fine-tune  │    │  Quantize   │    │  2× T4 (30GB)   │
└─────────────┘    └─────────────┘    └─────────────────┘
```

## 🚀 Quick Start

### 1. Extract Package
```bash
tar -xzf llcuda-v{VERSION}-cuda12-kaggle-t4x2.tar.gz
cd llcuda-v{VERSION}-cuda12-kaggle-t4x2
chmod +x bin/*
```

### 2. Start Multi-GPU Server
```bash
./bin/llama-server \\
    -m /path/to/model.gguf \\
    -ngl 99 \\
    --tensor-split 0.5,0.5 \\
    --split-mode layer \\
    -fa \\
    --host 0.0.0.0 \\
    --port 8080 \\
    -c 8192
```

### 3. Use with Python
```python
from llcuda.api import LlamaCppClient, kaggle_t4_dual_config

# Get optimal config for Kaggle
config = kaggle_t4_dual_config()
print(config.to_cli_args())

# Connect to server
client = LlamaCppClient("http://localhost:8080")

# OpenAI-compatible chat
response = client.chat.completions.create(
    messages=[{{"role": "user", "content": "Hello!"}}],
    max_tokens=100
)
print(response.choices[0].message.content)
```

## 📊 Multi-GPU Flags

| Flag | Description | Example |
|------|-------------|--------|
| `-ngl 99` | Offload all layers to GPU | Required |
| `--tensor-split` | VRAM ratio per GPU | `0.5,0.5` |
| `--split-mode` | Split strategy | `layer` or `row` |
| `--main-gpu` | Primary GPU ID | `0` |
| `-fa` | FlashAttention | Recommended |

## 📦 Recommended Models for 30GB VRAM

| Model | Quant | Size | Context | Fits? |
|-------|-------|------|---------|-------|
| Llama 3.1 70B | IQ3_XS | ~25GB | 4K | ✅ |
| Qwen2.5 32B | Q4_K_M | ~19GB | 8K | ✅ |
| Gemma 2 27B | Q4_K_M | ~16GB | 8K | ✅ |
| Llama 3.1 8B | Q8_0 | ~9GB | 16K | ✅ |
| Mistral 7B | Q8_0 | ~8GB | 32K | ✅ |

## 🔧 Unsloth → llcuda Workflow

```python
# 1. Fine-tune with Unsloth
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(...)
# ... training ...

# 2. Export to GGUF (Unsloth built-in)
model.save_pretrained_gguf("my_model", tokenizer, quantization_method="q4_k_m")

# 3. Run with llcuda
# ./bin/llama-server -m my_model-Q4_K_M.gguf -ngl 99 --tensor-split 0.5,0.5
```

## 📋 Build Info

- **llcuda Version:** {VERSION}
- **CUDA Version:** 12.4
- **Target GPU:** Tesla T4 × 2
- **Compute Capability:** SM 7.5 (Turing)
- **FlashAttention:** All quantization types
- **Build Date:** {BUILD_DATE}

## 📚 Resources

- [llcuda GitHub](https://github.com/llcuda/llcuda)
- [Unsloth](https://github.com/unslothai/unsloth)
- [llama.cpp](https://github.com/ggml-org/llama.cpp)
'''

with open(f"{PACKAGE_DIR}/README.md", "w") as f:
    f.write(readme_content)

print("✅ README.md created")
print(f"\n{readme_content[:1500]}...")

## Step 11: Create Helper Scripts

In [ ]:
# Create start-server.sh helper script
start_script = '''#!/bin/bash
# llcuda v2.2.0 - Start Multi-GPU Server
# Usage: ./start-server.sh <model.gguf> [port]

MODEL="$1"
PORT="${2:-8080}"

if [ -z "$MODEL" ]; then
    echo "Usage: $0 <model.gguf> [port]"
    echo "Example: $0 qwen2.5-7b-Q4_K_M.gguf 8080"
    exit 1
fi

SCRIPT_DIR="$(cd "$(dirname "$0")" && pwd)"

echo "Starting llama-server with dual T4 config..."
echo "Model: $MODEL"
echo "Port: $PORT"
echo ""

"$SCRIPT_DIR/bin/llama-server" \\
    --model "$MODEL" \\
    --n-gpu-layers 99 \\
    --tensor-split 0.5,0.5 \\
    --split-mode layer \\
    --flash-attn \\
    --host 0.0.0.0 \\
    --port "$PORT" \\
    --ctx-size 8192 \\
    --batch-size 2048 \\
    --ubatch-size 512 \\
    --parallel 4
'''

with open(f"{PACKAGE_DIR}/start-server.sh", "w") as f:
    f.write(start_script)
os.chmod(f"{PACKAGE_DIR}/start-server.sh", 0o755)

# Create quantize.sh helper script
quantize_script = '''#!/bin/bash
# llcuda v2.2.0 - Quantize Model
# Usage: ./quantize.sh <input.gguf> <output.gguf> [quant_type]

INPUT="$1"
OUTPUT="$2"
QUANT="${3:-Q4_K_M}"

if [ -z "$INPUT" ] || [ -z "$OUTPUT" ]; then
    echo "Usage: $0 <input.gguf> <output.gguf> [quant_type]"
    echo "Quant types: Q4_K_M (default), Q8_0, Q5_K_M, IQ4_XS, etc."
    exit 1
fi

SCRIPT_DIR="$(cd "$(dirname "$0")" && pwd)"

echo "Quantizing: $INPUT → $OUTPUT ($QUANT)"
"$SCRIPT_DIR/bin/llama-quantize" "$INPUT" "$OUTPUT" "$QUANT"
'''

with open(f"{PACKAGE_DIR}/quantize.sh", "w") as f:
    f.write(quantize_script)
os.chmod(f"{PACKAGE_DIR}/quantize.sh", 0o755)

print("✅ Helper scripts created:")
print("   - start-server.sh")
print("   - quantize.sh")

## Step 12: Create Distribution Archive

In [ ]:
import os
import hashlib

os.chdir("/kaggle/working")

TARBALL = f"{PACKAGE_NAME}.tar.gz"

print(f"Creating distribution archive: {TARBALL}")
print("="*60)

# Create tarball
!tar -czvf {TARBALL} {PACKAGE_NAME}

# Calculate SHA256
with open(TARBALL, "rb") as f:
    sha256 = hashlib.sha256(f.read()).hexdigest()

# Write checksum file
with open(f"{TARBALL}.sha256", "w") as f:
    f.write(f"{sha256}  {TARBALL}\n")

print("\n" + "="*60)
print("📦 DISTRIBUTION PACKAGE READY")
print("="*60)
!ls -lh {TARBALL}*
print(f"\nSHA256: {sha256}")

## Step 13: Test Multi-GPU Inference (Optional)

In [ ]:
# Download a small test model and verify multi-GPU works
from huggingface_hub import hf_hub_download
import subprocess
import time
import requests

print("Downloading small test model...")
model_path = hf_hub_download(
    repo_id="lmstudio-community/gemma-2-2b-it-GGUF",
    filename="gemma-2-2b-it-Q4_K_M.gguf",
    cache_dir="/kaggle/working/models"
)
print(f"✅ Model: {model_path}")

# Start server with multi-GPU
print("\nStarting llama-server with dual T4 config...")
server_cmd = [
    f"{PACKAGE_DIR}/bin/llama-server",
    "-m", model_path,
    "-ngl", "99",
    "--tensor-split", "0.5,0.5",
    "--split-mode", "layer",
    "-fa",
    "--host", "127.0.0.1",
    "--port", "8080",
    "-c", "4096"
]

print(f"Command: {' '.join(server_cmd)}")

server = subprocess.Popen(
    server_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT
)

# Wait for server
print("\nWaiting for server to start...")
for i in range(60):
    try:
        r = requests.get("http://127.0.0.1:8080/health", timeout=2)
        if r.status_code == 200:
            print(f"✅ Server ready in {i+1}s!")
            break
    except:
        time.sleep(1)
else:
    print("⚠️ Server startup timeout")

# Check GPU usage
print("\n📊 GPU Memory Usage:")
!nvidia-smi --query-gpu=index,memory.used,memory.total --format=csv

In [ ]:
# Test inference
import requests
import time

print("Testing multi-GPU inference...")
print("="*60)

start = time.time()
response = requests.post(
    "http://127.0.0.1:8080/v1/chat/completions",
    json={
        "messages": [{"role": "user", "content": "Explain quantum computing in 2 sentences."}],
        "max_tokens": 100,
        "temperature": 0.7
    },
    timeout=60
)
elapsed = time.time() - start

if response.status_code == 200:
    result = response.json()
    content = result["choices"][0]["message"]["content"]
    usage = result.get("usage", {})
    
    print(f"✅ Response ({elapsed:.2f}s):")
    print(f"   {content}")
    print(f"\n📊 Tokens: {usage.get('total_tokens', 'N/A')}")
    if usage.get('completion_tokens'):
        tps = usage['completion_tokens'] / elapsed
        print(f"📊 Speed: {tps:.1f} tokens/sec")
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

In [ ]:
# Cleanup - stop server
print("Stopping server...")
server.terminate()
server.wait()
print("✅ Server stopped")

# Show final GPU state
print("\n📊 Final GPU State:")
!nvidia-smi --query-gpu=index,memory.used,memory.total,utilization.gpu --format=csv

## Step 13b: Test Split-GPU Architecture (LLM + Graphistry)

In [ ]:
"""
Split-GPU Architecture Demo:
- GPU 0: llama-server (LLM inference)
- GPU 1: RAPIDS/Graphistry (graph simulation)
"""
import os
import subprocess
import time
import requests
import threading

print("="*70)
print("SPLIT-GPU ARCHITECTURE TEST")
print("="*70)

# ============================================================================
# GPU 0: Start llama-server (LLM)
# ============================================================================
print("\n🔧 GPU 0: Starting llama-server...")

# Force llama-server to use GPU 0 only
llama_env = os.environ.copy()
llama_env["CUDA_VISIBLE_DEVICES"] = "0"

server_cmd = [
    f"{PACKAGE_DIR}/bin/llama-server",
    "-m", model_path,
    "-ngl", "99",
    "-fa",
    "--host", "127.0.0.1",
    "--port", "8080",
    "-c", "4096"
]

server = subprocess.Popen(
    server_cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    env=llama_env
)

# Wait for server
for i in range(60):
    try:
        r = requests.get("http://127.0.0.1:8080/health", timeout=2)
        if r.status_code == 200:
            print(f"   ✅ llama-server ready on GPU 0 ({i+1}s)")
            break
    except:
        time.sleep(1)
else:
    print("   ⚠️ Server timeout")

# ============================================================================
# GPU 1: RAPIDS/Graphistry graph operations
# ============================================================================
print("\n🔧 GPU 1: Running RAPIDS graph simulation...")

# Force RAPIDS to use GPU 1 only
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import cudf
import cugraph

# Create sample graph data (simulating knowledge graph from LLM)
edges = cudf.DataFrame({
    "src": [0, 1, 2, 3, 4, 0, 1, 2],
    "dst": [1, 2, 3, 4, 0, 2, 3, 4],
    "weight": [1.0, 2.0, 1.5, 0.5, 3.0, 2.5, 1.0, 0.8]
})

# Create cuGraph graph
G = cugraph.Graph()
G.from_cudf_edgelist(edges, source="src", destination="dst", edge_attr="weight")

print(f"   Graph: {G.number_of_vertices()} vertices, {G.number_of_edges()} edges")

# Run PageRank on GPU 1
pagerank = cugraph.pagerank(G)
print(f"   PageRank computed: {len(pagerank)} nodes")
print(f"   Top node: {pagerank.nlargest(1, 'pagerank')['vertex'].values[0]}")

# ============================================================================
# Combined workflow: LLM query → Graph update
# ============================================================================
print("\n🔗 Combined LLM + Graph workflow...")

# Reset CUDA_VISIBLE_DEVICES for requests
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

# Query LLM on GPU 0
response = requests.post(
    "http://127.0.0.1:8080/v1/chat/completions",
    json={
        "messages": [{"role": "user", "content": "List 3 related concepts to 'machine learning'"}],
        "max_tokens": 100
    },
    timeout=30
)

if response.status_code == 200:
    llm_output = response.json()["choices"][0]["message"]["content"]
    print(f"   LLM (GPU 0): {llm_output[:100]}...")
    
    # Simulate adding LLM-derived edges to graph
    new_edges = cudf.DataFrame({
        "src": [5, 5, 5],
        "dst": [0, 1, 2],
        "weight": [1.0, 1.0, 1.0]
    })
    all_edges = cudf.concat([edges, new_edges])
    G2 = cugraph.Graph()
    G2.from_cudf_edgelist(all_edges, source="src", destination="dst", edge_attr="weight")
    print(f"   Graph (GPU 1): Updated to {G2.number_of_vertices()} vertices")

print("\n📊 GPU Memory Usage:")
!nvidia-smi --query-gpu=index,name,memory.used,memory.total --format=csv

# Cleanup
server.terminate()
server.wait()
print("\n✅ Split-GPU test complete!")

## Step 14: Final Summary

In [ ]:
import os
os.chdir("/kaggle/working")

print("="*70)
print("🎉 llcuda v2.2.0 BUILD COMPLETE!")
print("="*70)

print(f"\n📦 Distribution Package:")
!ls -lh {PACKAGE_NAME}.tar.gz

print(f"\n📁 Package Contents:")
!ls -la {PACKAGE_NAME}/

print(f"\n🔧 Binaries:")
!ls -lh {PACKAGE_NAME}/bin/ | head -10

print(f"\n📋 Metadata Summary:")
print(f"   Version: {VERSION}")
print(f"   Platform: Kaggle 2× Tesla T4")
print(f"   CUDA: {cuda_version}")
print(f"   Compute: SM 7.5 (Turing)")
print(f"   FlashAttention: ✅ All quants")
print(f"   Multi-GPU: ✅ Native CUDA")

print(f"\n🚀 Next Steps:")
print(f"   1. Download: {PACKAGE_NAME}.tar.gz")
print(f"   2. Extract: tar -xzf {PACKAGE_NAME}.tar.gz")
print(f"   3. Run: ./start-server.sh model.gguf 8080")

print(f"\n📥 Download from Kaggle Output tab")
print(f"   or copy to output: !cp {PACKAGE_NAME}.tar.gz /kaggle/output/")

In [ ]:
# Copy to Kaggle output for download
import shutil

os.makedirs("/kaggle/output", exist_ok=True)
shutil.copy(f"/kaggle/working/{PACKAGE_NAME}.tar.gz", "/kaggle/output/")
shutil.copy(f"/kaggle/working/{PACKAGE_NAME}.tar.gz.sha256", "/kaggle/output/")

print("✅ Package copied to /kaggle/output/ for download")
!ls -lh /kaggle/output/